In [16]:
import psycopg2
import pandas as pd
import numpy as np
import networkx as nx

In [17]:
# connect to database
conn = psycopg2.connect(
    host="localhost",
    port=5450,
    database="postgres",
    user="postgres",
    password="postgres"
)

# create cursor
cur = conn.cursor()

In [18]:
# get the topics table
cur.execute("SELECT * FROM topics")
topics = cur.fetchall()

# get the columns
cols = [desc[0] for desc in cur.description]

# create dataframe
topics_df = pd.DataFrame(topics, columns=cols)

topics_df.head(15)

,topic
0,infrastructure access is a crucial aspect of m...
1,communication
2,energy
3,and water supply. however
4,the current approach to infrastructure access ...
5,the demand for infrastructure services increas...
6,extreme weather events
7,and other climate-related phenomena pose signi...
8,the digital revolution has transformed the way...
9,there is a need to address issues of equity an...


In [19]:
topics_df.tail(15)

,topic
62829,rubrik polaris ui
62830,file properties
62831,file change rate
62832,inconsistent content
62833,entropy change
62834,custom-trained machine learning model
62835,rubrik’s polaris data repository
62836,unified system of recor
62837,anomaly analysis
62838,radar workspace


In [20]:
# import data into pandas dataframe
cur.execute("SELECT * FROM resources")

# get column names
colnames = [desc[0] for desc in cur.description]

# create dataframe
df = pd.DataFrame(cur.fetchall(), columns=colnames)

In [21]:
# print dataframe
df.head()

,url,hid,heading,hnum,next_heading,subheading,content,topics
0,https://adaptive.live/blog/rethink-infrastruct...,root,Why the world needs to rethink Infrastructure ...,0,1.0,NaN,,"[""infrastructure access is a crucial aspect of..."
1,https://adaptive.live/blog/rethink-infrastruct...,h2,The Increasing Data Breach Problem,1,NaN,2.0,The problem of cyber breaches is only getting ...,"[""data breach problem"", ""cyber breaches"", ""tre..."
2,https://adaptive.live/blog/rethink-infrastruct...,h2,Insider Threat: a new problem within organizat...,2,NaN,3.0,"Recently, a new type of threat has started imp...","[""insider threats"", ""cyber security risk"", ""or..."
3,https://adaptive.live/blog/rethink-infrastruct...,h2,Complexity of the Modern-day Infrastructure,3,4.0,8.0,In order to understand the problems with infra...,"[""complexity"", ""modern-day infrastructure"", ""p..."
4,https://adaptive.live/blog/rethink-infrastruct...,h3,Remote-first trend post-covid,4,NaN,5.0,"In March 2020 Covid struck, and remote work we...","[""remote work"", ""covid"", ""mainstream"", ""rushed..."


In [22]:
df.tail()

,url,hid,heading,hnum,next_heading,subheading,content,topics
9369,https://www.rubrik.com/blog/products-solutions...,h2,Dealing with Ransomware,1,NaN,2.0,No one likes having their possessions held for...,"[""dealing with ransomware"", ""data center"", ""in..."
9370,https://www.rubrik.com/blog/products-solutions...,h2,Rubrik Polaris Radar,2,NaN,3.0,Radar works in concert with a layered approach...,"[""radar"", ""layered approach to security"", ""dat..."
9371,https://www.rubrik.com/blog/products-solutions...,h2,Anomaly Detection,3,NaN,4.0,Radar uses machine learning to understand beha...,"[""anomaly detection"", ""machine learning"", ""beh..."
9372,https://www.rubrik.com/blog/products-solutions...,h2,Anomaly Analysis,4,NaN,5.0,"Once an anomaly is detected, an analysis of wh...","[""anomaly analysis"", ""files"", ""applications"", ..."
9373,https://www.rubrik.com/blog/products-solutions...,h2,Faster Recovery,5,6.0,NaN,"As we at Rubrik like to say, “the most effecti...","[""rubrik"", ""ransomware"", ""defense in depth"", ""..."


In [23]:
# print length of dataframe
len(df)

9374

In [24]:
# fetch the urls from the database
cur.execute("SELECT * FROM urls")
urls = cur.fetchall()

# get column names
colnames = [desc[0] for desc in cur.description]

# close cursor
cur.close()

# create dataframe
urls_df = pd.DataFrame(urls, columns=colnames)

# print dataframe
urls_df.head()

,url,domain,author,pubdate
0,https://adaptive.live/blog/rethink-infrastruct...,adaptive.live,ronak massand,08142023
1,https://adaptive.live/blog/adaptive-has-comple...,adaptive.live,debarshi basak,07182023
2,https://adaptive.live/blog/bits-and-breaches-s...,adaptive.live,ronak massand,07122023
3,https://adaptive.live/blog/bits-and-breaches-t...,adaptive.live,ronak massand,06222023
4,https://adaptive.live/blog/evolution-of-intern...,adaptive.live,debarshi basak,06152023


In [25]:
urls_df.tail()

,url,domain,author,pubdate
1182,https://www.rubrik.com/blog/products-solutions...,www.rubrik.com,kapil chhabra,10102018
1183,https://www.rubrik.com/blog/customers/18/10/je...,www.rubrik.com,rubrik logo,10052018
1184,https://www.rubrik.com/blog/technology/18/10/r...,www.rubrik.com,chris wahl,10032018
1185,https://www.rubrik.com/blog/products-solutions...,www.rubrik.com,pierre-francois guglielmi,09272018
1186,https://www.rubrik.com/blog/products-solutions...,www.rubrik.com,rubrik logo,09242018


In [26]:
# len of urls with domain = www.strongdm.com
len(urls_df[urls_df['domain'] == 'www.rubrik.com'])

381

In [27]:
# len of unique urls with domain = www.strongdm.com
len(urls_df[urls_df['domain'] == 'www.rubrik.com']['url'].unique())

381

In [ ]:
kwlen = 4